# Dataflow using skill return values and parameters

This example notebook demonstrates moving over a couple of objects:

- Use `estimate_pose` skill to detect objects
- Pass information retrieved as return value from one skill as a parameter to another skill

In particular, this example covers:

- Behavior tree features
    - Skill return values
    - Passing data to a skill using a blackboard value for parameters
    - Use the loop counter of a `Loop` node
    - Write a custom condition expression

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [flowstate.intrinsic.ai](https://flowstate.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `building_block:building_block_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `building_block:building_block_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

First, connect to your solution and define convenience shortcuts:

In [ ]:
from intrinsic.math.python import data_types
from intrinsic.solutions import behavior_tree as bt
from intrinsic.solutions import deployments
from intrinsic.solutions import worlds

solution = deployments.connect_to_selected_solution()

executive = solution.executive
world = solution.world
skills = solution.skills
resources = solution.resources

move_robot = skills.ai.intrinsic.move_robot
estimate_pose = skills.ai.intrinsic.estimate_pose

# Create the Behavior Tree

First, define most of the necessary skills. The overall flow is to move to a home position, then identify blocks in the workcell, and then move to the first one seen.

In [ ]:
move_home = bt.Task(
    name="Move to Home",
    action=move_robot(
        motion_segments=[
            move_robot.intrinsic_proto.skills.MotionSegment(
                joint_position=world.robot.joint_configurations.home,
                motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
            )
        ],
        arm_part=world.robot,
    ),
)

move_to_view = bt.Task(
    name="Move to view",
    action=move_robot(
        motion_segments=[
            move_robot.intrinsic_proto.skills.MotionSegment(
                joint_position=move_robot.intrinsic_proto.icon.JointVec(
                    # Known configuration in which the pose estimator has been trained
                    joints=[-1.176, -1.934, -1.897, -0.746, 2.199, 0.922]
                ),
                motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
            )
        ],
        arm_part=world.robot,
    ),
)

Now, for using the return values. Return values are available as a key-value map located in the executive. When creating your application, you will only have to interact with the keys of the values, as the actual values will be computed during execution and available using the key.

Every skill you create has a property `result` which can be used to later access the result value. You can also use auto-completion on this, even if the value is not yet available.
Instantiate the estimate pose skill and have a look.

In [ ]:
estimate_blocks = bt.Task(
    name="Detect block",
    action=estimate_pose(
        camera=resources.wrist_camera,
        pose_estimator=solution.pose_estimators.building_block_ml_estimator,
        min_num_instances=1,
        max_num_instances=6,
        # Raise object position by 4cm to avoid collision when using as approach pose
        object_t_target=data_types.Pose3(translation=[0, 0, -0.04]),
    ),
)

Using the following coude, you can print the generated name for the result value of the skill. It will be used during execution to look up the actual pose values.

In [ ]:
estimate_blocks.result.value_access_path()

Next, since you want to move over all available blocks use a loop node.

In [ ]:
loop_node = bt.Loop(name="Iterate over Blocks")

Since you want to iterate over the available blocks you can use the loop counter to access a value in the estimate array. <br>
Again, the loop counter is only a string value denoting the key of the index in the key value map. During execution this key will be used to look up the current value of the loop counter. It will only be available inside the loop and is cleared as soon as the loop exits.

In [ ]:
loop_node.loop_counter

Now, put everything together by creating a target for and instance of the `move_robot` skill, which takes the pose from the `estimate_connectors` skill and uses the loop counter as an index in the list.

In [ ]:
block_target = move_robot.intrinsic_proto.world.geometric_constraints.PoseEquality(
    moving_frame=world.picobot_gripper.tool_frame,
    target_frame=world.root,
    # The following resolves to a blackboard value and is thus computed at execution time
    target_frame_offset=estimate_blocks.result.estimates[
        loop_node.loop_counter
    ].root_t_target,
)
move_to_block = bt.Task(
    name="Move to block",
    action=move_robot(
        motion_segments=[
            move_robot.intrinsic_proto.skills.MotionSegment(
                cartesian_pose=block_target,
                motion_type=move_robot.intrinsic_proto.skills.MotionSegment.JOINT,
            )
        ],
        arm_part=world.robot,
    ),
)

Now we can fully configure the loop. We set a condition that limits iteration to the detected objects, such that the loop counter (used as index into the pose estimator result list) does not run out of bounds. The condition is given as a [CEL](https://github.com/google/cel-spec/blob/master/doc/intro.md) expression using the identifiers we inspected before. The condition will be evaluated once before each loop iteration, like in a programming language's while statement.

In [ ]:
loop_node.set_while_condition(
    bt.Blackboard(
        f"size({estimate_blocks.result.estimates.value_access_path()}) > {loop_node.loop_counter}"
    )
)
loop_node.set_do_child(move_to_block)

Now we put the created nodes together into a sequence for the tree. We can visualize the created tree.

In [ ]:
my_bt = bt.BehaviorTree(
    root=bt.Sequence([move_home, move_to_view, estimate_blocks, loop_node])
)
my_bt.show()

# Run Behavior Tree

In [ ]:
executive.run(my_bt)

## Inspect return value of pose estimation skill

The `estimate_pose` skill returns a list of estimates for recognized objects. After execution, inspect the result value of the `estimate_blocks` instance of that skill.

In [ ]:
executive.get_value(estimate_blocks.result)